Import packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Read xlsx data

In [ ]:
#set up strings with directory/file names
file_dir = "C:/Users/hamilton.w/Documents/MCM_data/"
data_name = "MCM_NFLIS_data.xlsx"

#read the xlsx, and convert to a pandas dataframe
#some versions of pandas requires the optional argument ``sheet_name'' instead of ``sheetname''
df = pd.read_excel("{}{}".format(file_dir,data_name), sheetname = "Data")

Initial summary of what's in the spreadsheet

In [ ]:
#print the column names
print(df.columns)

#summarize the entries in each column
for label in df.columns:
	print(label)
	print(df[label].unique())

Summary of the entries, sorted by drug reports

In [ ]:
#sort all entries by number of reported cases in state
sort_order_states = np.argsort(df.iloc[:,-3].values)
df.iloc[sort_order_states,[0,1,2,6,7,8,9]][:100]

Same summary, just order reversed

In [ ]:
#reverse sort order
sort_order_states = sort_order_states[::-1]
df.iloc[sort_order_states,[0,1,2,6,7,8,9]][:50]

Isolate a dataframe with the state drug reports

In [ ]:
#sort state drug reports
df_state_totals = df.iloc[:,[0,1,3,9]]
df_state_totals = df_state_totals.drop_duplicates()
df_state_totals.sort_values("TotalDrugReportsState")[::-1]

Isolate a dataframe with the county drug reports

In [ ]:
#sort county drug reports
df_county_totals = df.iloc[:,[0,1,2,4,8,9]]
df_county_totals = df_county_totals.drop_duplicates()
df_county_totals.sort_values("TotalDrugReportsCounty")[::-1]

Isolate the heroin and fentanyl data

In [ ]:
#get heroin data 
df_heroin = df[df["SubstanceName"] == "Heroin"]
df_heroin = df_heroin.iloc[:,[0,1,3,7]]

#get fentanyl data
df_fentanyl = df[df["SubstanceName"] == "Fentanyl"]
df_fentanyl = df_fentanyl.iloc[:,[0,1,3,7]]

Get totals by year

In [ ]:
#get totals by year 
years = df["YYYY"].unique()
heroin_totals = np.zeros(len(years))
fentanyl_totals = np.zeros(len(years))

for i in range(len(years)):
	y = years[i]
	heroin_totals[i] = sum(df_heroin[df_heroin["YYYY"]==y].iloc[:,-1].values)
	
	fentanyl_totals[i] = sum(df_fentanyl[df_fentanyl["YYYY"]==y].iloc[:,-1].values)

Plot the drug report rates

In [ ]:
plt.plot(years,heroin_totals, c = "y")
plt.plot(years,fentanyl_totals, c = "b")

plt.ylabel("Total drug reports")
plt.xlabel("Year")
plt.show()

Read in the meta data for 2016

In [ ]:
#read meta data 
se_data_name = "ACS_16_5YR_DP02_with_ann.csv"
se_metadata_name = "ACS_16_5YR_DP02_metadata.csv"

df_se = pd.read_csv("{}{}".format(file_dir,se_data_name))
df_se_meta = pd.read_csv("{}{}".format(file_dir,se_metadata_name))

See what's in each csv

In [ ]:
df_se.head()

In [ ]:
df_se_meta.head()

Isolate the dictionary for numbers of households

In [ ]:
#isolate the house dictionary
df_se.iloc[:,[1,3]]

#check that the sizes match up 
print(len(df_se["GEO.id2"].unique()))
print(len(df["FIPS_Combined"].unique()))

Sizes don't match up... where?

In [ ]:
#convert the geo_ids to ints
orig_data_locs = df["FIPS_Combined"].unique()
unique_meta_locs = df_se["GEO.id2"].iloc[1:].unique().astype(int)

#figure out which counties are different 
print(list(set(orig_data_locs) - set(unique_meta_locs)))

#double check 
print(sum(df_se["GEO.id2"]=="51515"))

#which city?
print(df[df["FIPS_Combined"]==51515])

Let's normalize our original drug reports by numbers of households

In [ ]:
#recover state total households
state_FIPS_dict = {}
state_total_households = {}

#set up what we need for the state households totals
for s in df["State"].unique():
	#get the state FIPS 
	state_FIPS = df[df["State"]==s]["FIPS_State"].iloc[0]
	
	state_FIPS_dict[s] = state_FIPS
	state_total_households[state_FIPS] = 0.

#actually compute the estimated state total households
for i in range(1,len(df_se)):
	geo_id = df_se["GEO.id2"].iloc[i]
	num_households = df_se["HC01_VC03"].iloc[i]
	
	state_total_households[int(geo_id[:2])] += int(num_households)

print(state_FIPS_dict)
print(state_total_households)

Make a copy of the original dataframe

In [ ]:
#copy the df
normed_df = df.copy()

Convert the state totals in the copied dataframe

In [ ]:
#convert the state totals 
for s in df["State"].unique():
	s_FIPS = state_FIPS_dict[s]
	transform_func = lambda x: x/state_total_households[s_FIPS]
	
	normed_df.loc[normed_df["State"]==s,"TotalDrugReportsState"] = normed_df[normed_df["State"]==s]["TotalDrugReportsState"].apply(transform_func)

Convert the county totals

In [ ]:
#convert the county totals 
for c in unique_meta_locs:
	county_total = df_se[df_se["GEO.id2"]==str(c)]["HC01_VC03"].values
	transform_func = lambda x: x/float(county_total)
	
	normed_df.loc[normed_df["FIPS_Combined"]==c,"TotalDrugReportsCounty"] = normed_df[normed_df["FIPS_Combined"]==c]["TotalDrugReportsCounty"].apply(transform_func)

Cut out the county without the relevant data

In [ ]:
#remove 51515
#get indices of 51515 entries 
drop_indices = np.arange(len(normed_df))[normed_df["FIPS_Combined"]==51515]

normed_df = normed_df.drop(drop_indices)

Repeat the same analysis as before, with the normalized data

In [ ]:
#sort state drug reports
df_state_totals = normed_df.iloc[:,[0,1,3,9]]
df_state_totals = df_state_totals.drop_duplicates()
df_state_totals.sort_values("TotalDrugReportsState")[::-1]

In [ ]:
#sort county drug reports
df_county_totals = normed_df.iloc[:,[0,1,2,8]]
df_county_totals = df_county_totals.drop_duplicates()
df_county_totals.sort_values("TotalDrugReportsCounty")[::-1]

What are the households and populations for the new top counties?

In [ ]:
#manually write out the pairs we want to look at
state_county_pairs = [("VA","NELSON"),("KY","BELL"),("OH","HAMILTON"),("KY","PERRY")]

#for each state and county pair
for sc_pair in state_county_pairs:
    #save the local dataframe
    temp_df = df[(df["State"]==sc_pair[0])&(df["COUNTY"]==sc_pair[1])]
    
    #[0,1,2,8,9] are the year, state, county, total county reports, total state reports
    print(temp_df.iloc[:,[0,1,2,8,9]].drop_duplicates())
    
    #print the 2016 housing data for each state, county pair 
    temp_FIPS = temp_df["FIPS_Combined"].iloc[0]
    print("The number of households is:")
    print(float(df_se[df_se["GEO.id2"] == str(temp_FIPS)]["HC01_VC03"]))
